<a href="https://colab.research.google.com/github/jinseriouspark/embedding_for_all/blob/main/%5Bw2%5D_fine_tuning_%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fine Tuning

In [41]:
!pip install datasets
!pip install transformers[torch]
!pip install accelrate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [1]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric

## load dataset

In [2]:
data = load_dataset('wikiann','ko')
label_names = data['train'].features['ner_tags'].feature.names

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
data.shape, label_names

({'validation': (10000, 4), 'test': (10000, 4), 'train': (20000, 4)},
 ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])

In [4]:
# token과 ner-tag
data['train']

Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'spans'],
    num_rows: 20000
})

In [5]:
data['train'][0]

{'tokens': ['현재',
  '대한민국',
  'K리그',
  '챌린지의',
  '서울',
  '이랜드',
  'FC에서',
  '활약하고',
  '있다',
  '.'],
 'ner_tags': [0, 5, 3, 4, 3, 4, 4, 0, 0, 0],
 'langs': ['ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko'],
 'spans': ['LOC: 대한민국', 'ORG: K리그 챌린지의', 'ORG: 서울 이랜드 FC에서']}

## load model : koelectra
: pretrained ELECTRA language model 한국어판

- 생성기에서 토큰을 보고 판별기에서 그것이 '진짜' 토큰인지 '가짜 ' 토큰인지 판단하여 학습함
- 이 방법을 사용할 경우, 모든 입력 토큰을 훈련할 수 있으며, bert 등과 비교하여 효과가 우수함
- WordPiece를 사용



(참고자료)
- 링크 1 : https://huggingface.co/monologg/koelectra-base-v3-generator
- 링크 2 : https://github.com/monologg/KoELECTRA/blob/master/README_EN.md
- 링크 3 : https://github.com/monologg/KoELECTRA/blob/master/finetune/README_EN.md

In [6]:
model_name = "monologg/koelectra-small-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name,
                                                       num_labels = len(label_names)).to('cuda')


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
print(data['train'][0])
text =' '.join(data['train']['tokens'][0])
print(tokenizer.encode(text))

{'tokens': ['현재', '대한민국', 'K리그', '챌린지의', '서울', '이랜드', 'FC에서', '활약하고', '있다', '.'], 'ner_tags': [0, 5, 3, 4, 3, 4, 4, 0, 0, 0], 'langs': ['ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko'], 'spans': ['LOC: 대한민국', 'ORG: K리그 챌린지의', 'ORG: 서울 이랜드 FC에서']}
[2, 6339, 7001, 47, 19611, 18756, 4234, 6265, 23358, 10839, 4073, 4129, 8377, 4279, 4219, 3249, 4176, 18, 3]


In [8]:
# batch 단위로 인코딩 하는 함수
batch_result_line2 = tokenizer.batch_encode_plus(data['train']['tokens'][:3], is_split_into_words = True)
batch_result_line2

{'input_ids': [[2, 6339, 7001, 47, 19611, 18756, 4234, 6265, 23358, 10839, 4073, 4129, 8377, 4279, 4219, 3249, 4176, 18, 3], [2, 11, 11, 11, 3598, 25892, 11, 11, 11, 3], [2, 11554, 4172, 4162, 2780, 4031, 3311, 4112, 2241, 15, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [9]:
# input_ids, token_type_ids, attention_mask를 위한 값 생성

def tokenize_adjust_labels(all_samples):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples['tokens'],
                                                  is_split_into_words = True)

  total_adjusted_labels = []
  for k in range(0, len(tokenized_samples['input_ids'])):
    prev_wid = -1
    word_id_list = tokenized_samples.word_ids(batch_index = k) # output # 같은 단어는 같은 id : [None, 0, 1, 2, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 8, 8, 9, None]
    existing_label_ids = all_samples['ner_tags'][k] # output : [[0, 5, 3, 4, 3, 4, 4, 0, 0, 0],...]
    i = -1
    adjusted_label_ids = []

    for wid in word_id_list:
      if wid is None: # 단어 리스트에 포함되지 않으면 -100
        adjusted_label_ids.append(-100)
      elif wid != prev_wid: # 새 단어가 등장하여 과거 word_id 와 다를 경우
        i += 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else: # 등장하기 시작한 word_id 가 유지될 경우
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples['labels'] = total_adjusted_labels
  return tokenized_samples


In [10]:
tokenized_dataset = data.map(tokenize_adjust_labels, batched = True)

In [11]:
data # 이전

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})

In [12]:
tokenized_dataset # tokenizer.tokenize()를 통과시켰을 때 및 label이 추가됨

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [14]:
!pip install seqeval

metric = load_metric('seqeval')

<ipython-input-14-fb2632c3f8b2>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('seqeval')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
def compute_metrics(pred):
  predictions, labels = pred
  predictions = np.argmax(predictions, axis=2)
  print(predictions)

  # speical token은 삭제
  true_predictions = [
      [label_names[p] for (p, l) in zip(prediction, label) if l != -100
       for prediction, label in zip(predictions, labels)]
  ]

  true_labels = [
      [label_names[l] for (p, l) in zip(prediction, label) if l != -100
       for prediction, label in zip(predictions, labels)]
  ]

  # metrics.compute
  results = metric.compute(predictions = true_predictions, references = true_labels)
  print(results)
  flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
  }

  for k in results.keys():
    if k not in flattened_results.keys():
      flattened_results[f'{k}_f1'] = results[k]['f1']

  return flattened_results

In [ ]:
# 추상화된 객체 사용해서 학습
training_args = TrainingArguments(
    output_dir = './find_tune_bert_output',
    evaluation_strategy='steps', # 훈련중에 채택할 평가전략 (no, steps, epoch)
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 7,
    weight_decay = 0.01,
    logging_steps = 1000,
    save_strategy = 'no', # 훈련중에는 저장하지 않음,
    use_cpu = False
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss
